In [6]:
%pip install mysql-connector-python
%pip install faker
%pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.5 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import streamlit as st
from openpyxl import Workbook
import warnings
warnings.filterwarnings('ignore')


In [19]:
fake = Faker()
np.random.seed(42)
random.seed(42)

# Generate synthetic e-commerce data similar to Meesho
def generate_ecommerce_data(num_orders=10000):
    data = []

    # Product categories relevant to Meesho
    categories = ['Fashion', 'Beauty', 'Home & Kitchen', 'Electronics', 'Kids', 'Bags']
    cities = ['Bangalore', 'Delhi', 'Mumbai', 'Chennai', 'Hyderabad', 'Kolkata', 'Pune', 'Ahmedabad']

    start_date = datetime(2024, 1, 1)
    end_date = datetime(2025, 8, 30)

    for i in range(num_orders):
        order_date = fake.date_between(start_date=start_date, end_date=end_date)

        # Generate realistic business patterns
        base_price = np.random.exponential(300) + 50  # Price distribution
        quantity = np.random.choice([1, 2, 3, 4], p=[0.6, 0.25, 0.1, 0.05])
         # Weekend boost
        if order_date.weekday() >= 5:
            base_price *= 1.1

        order_data = {
            'order_id': f'ORD{str(i+1).zfill(6)}',
            'order_date': order_date,
            'customer_id': f'CUST{random.randint(1, 3000)}',
            'product_id': f'PROD{random.randint(1, 500)}',
            'product_category': random.choice(categories),
            'quantity': quantity,
            'unit_price': round(base_price, 2),
            'total_price': round(base_price * quantity, 2),
            'customer_city': random.choice(cities),
            'discount_percent': random.choice([0, 5, 10, 15, 20]),
            'payment_method': random.choice(['UPI', 'Credit_Card', 'Debit_Card', 'COD']),
            'delivery_days': random.randint(2, 7)
        }
        # Apply discount
        discount_amount = order_data['total_price'] * (order_data['discount_percent'] / 100)
        order_data['final_price'] = round(order_data['total_price'] - discount_amount, 2)
        order_data['discount_amount'] = round(discount_amount, 2)

        data.append(order_data)

    return pd.DataFrame(data)

    # Generate the dataset
df = generate_ecommerce_data(10000)
print("Dataset created successfully!")
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())



Dataset created successfully!
Dataset shape: (10000, 14)

First 5 rows:
    order_id  order_date customer_id product_id product_category  quantity  \
0  ORD000001  2024-05-20    CUST2620     PROD58          Fashion         4   
1  ORD000002  2025-03-13     CUST420    PROD347             Bags         1   
2  ORD000003  2024-12-12     CUST123     PROD48           Beauty         1   
3  ORD000004  2024-11-27     CUST815    PROD367             Bags         3   
4  ORD000005  2025-05-27    CUST1140    PROD415          Fashion         2   

   unit_price  total_price customer_city  discount_percent payment_method  \
0      190.78       763.12     Hyderabad                 5    Credit_Card   
1      445.02       445.02         Delhi                20            COD   
2      100.89       100.89       Chennai                20            UPI   
3       67.95       203.85          Pune                 5            COD   
4      325.72       651.45        Mumbai                15     Debit_Card 

In [21]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect('meesho_analytics.db')

# Save DataFrame to database
df.to_sql('orders', conn, if_exists='replace', index=False)

print("Data saved to SQLite database!")


Data saved to SQLite database!


In [22]:
# Function to execute SQL queries
def execute_query(query, description=""):
    print(f"\n{'='*50}")
    print(f"Query: {description}")
    print(f"{'='*50}")
    print(f"SQL: {query}")
    print(f"{'-'*50}")

    result = pd.read_sql_query(query, conn)
    print(result)
    return result

# 1. Monthly Revenue Trend with Growth Rate
monthly_revenue_query = """
WITH monthly_stats AS (
    SELECT
        strftime('%Y-%m', order_date) as month,
        COUNT(*) as total_orders,
        SUM(final_price) as total_revenue,
        AVG(final_price) as avg_order_value,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM orders
    GROUP BY strftime('%Y-%m', order_date)
),
revenue_growth AS (
    SELECT *,
        LAG(total_revenue) OVER (ORDER BY month) as prev_month_revenue,
        ROUND(
            ((total_revenue - LAG(total_revenue) OVER (ORDER BY month)) * 100.0 /
            LAG(total_revenue) OVER (ORDER BY month)), 2
        ) as revenue_growth_pct
    FROM monthly_stats
)
SELECT * FROM revenue_growth ORDER BY month;
"""

monthly_data = execute_query(monthly_revenue_query, "Monthly Revenue Trend with Growth Rate")

# 2. Customer Segmentation by Purchase Behavior
customer_segmentation_query = """
WITH customer_metrics AS (
    SELECT
        customer_id,
        COUNT(*) as order_frequency,
        SUM(final_price) as total_spent,
        AVG(final_price) as avg_order_value,
        MAX(order_date) as last_order_date,
        MIN(order_date) as first_order_date,
        julianday('2025-08-30') - julianday(MAX(order_date)) as days_since_last_order
    FROM orders
    GROUP BY customer_id
)
SELECT
    CASE
        WHEN total_spent >= 5000 AND order_frequency >= 5 THEN 'High Value'
        WHEN total_spent >= 2000 AND order_frequency >= 3 THEN 'Medium Value'
        WHEN days_since_last_order <= 30 THEN 'Recent Active'
        WHEN days_since_last_order > 90 THEN 'At Risk'
        ELSE 'Regular'
    END as customer_segment,
    COUNT(*) as customer_count,
    ROUND(AVG(total_spent), 2) as avg_total_spent,
    ROUND(AVG(order_frequency), 2) as avg_order_frequency
FROM customer_metrics
GROUP BY customer_segment
ORDER BY customer_count DESC;
"""

customer_segments = execute_query(customer_segmentation_query, "Customer Segmentation Analysis")

# 3. Product Performance Analysis with Statistical Insights
product_performance_query = """
SELECT
    product_category,
    COUNT(*) as total_orders,
    SUM(quantity) as total_quantity_sold,
    ROUND(SUM(final_price), 2) as total_revenue,
    ROUND(AVG(final_price), 2) as avg_order_value,
    ROUND(AVG(discount_percent), 2) as avg_discount_rate,
    COUNT(DISTINCT customer_id) as unique_customers,
    ROUND(SUM(final_price) * 100.0 / (SELECT SUM(final_price) FROM orders), 2) as revenue_share_pct
FROM orders
GROUP BY product_category
ORDER BY total_revenue DESC;
"""

product_performance = execute_query(product_performance_query, "Product Category Performance")

# 4. City-wise Business Analysis
city_analysis_query = """
SELECT
    customer_city,
    COUNT(*) as total_orders,
    ROUND(SUM(final_price), 2) as total_revenue,
    ROUND(AVG(final_price), 2) as avg_order_value,
    COUNT(DISTINCT customer_id) as unique_customers,
    ROUND(AVG(delivery_days), 1) as avg_delivery_days,
    ROUND(
        SUM(final_price) / COUNT(DISTINCT customer_id), 2
    ) as revenue_per_customer
FROM orders
GROUP BY customer_city
ORDER BY total_revenue DESC;
"""

city_analysis = execute_query(city_analysis_query, "City-wise Business Performance")



Query: Monthly Revenue Trend with Growth Rate
SQL: 
WITH monthly_stats AS (
    SELECT 
        strftime('%Y-%m', order_date) as month,
        COUNT(*) as total_orders,
        SUM(final_price) as total_revenue,
        AVG(final_price) as avg_order_value,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM orders
    GROUP BY strftime('%Y-%m', order_date)
),
revenue_growth AS (
    SELECT *,
        LAG(total_revenue) OVER (ORDER BY month) as prev_month_revenue,
        ROUND(
            ((total_revenue - LAG(total_revenue) OVER (ORDER BY month)) * 100.0 / 
            LAG(total_revenue) OVER (ORDER BY month)), 2
        ) as revenue_growth_pct
    FROM monthly_stats
)
SELECT * FROM revenue_growth ORDER BY month;

--------------------------------------------------
      month  total_orders  total_revenue  avg_order_value  unique_customers  \
0   2024-01           522      261365.14       500.699502               479   
1   2024-02           479      233926.97       488

In [23]:
# KPI Calculations with Statistical Analysis
def calculate_kpis(df):
    kpis = {}

    # Basic KPIs
    kpis['total_revenue'] = df['final_price'].sum()
    kpis['total_orders'] = len(df)
    kpis['avg_order_value'] = df['final_price'].mean()
    kpis['unique_customers'] = df['customer_id'].nunique()
    kpis['revenue_per_customer'] = kpis['total_revenue'] / kpis['unique_customers']

    # Advanced Statistical KPIs
    kpis['aov_std'] = df['final_price'].std()
    kpis['aov_median'] = df['final_price'].median()
    kpis['revenue_skewness'] = stats.skew(df['final_price'])
    kpis['revenue_kurtosis'] = stats.kurtosis(df['final_price'])

    # Business-specific KPIs
    kpis['avg_discount_rate'] = df['discount_percent'].mean()
    kpis['total_discount_given'] = df['discount_amount'].sum()
    kpis['avg_delivery_days'] = df['delivery_days'].mean()

    # Conversion and engagement metrics
    repeat_customers = df.groupby('customer_id').size()
    kpis['repeat_customer_rate'] = (repeat_customers > 1).sum() / len(repeat_customers)
    kpis['avg_orders_per_customer'] = repeat_customers.mean()

    return kpis

kpis = calculate_kpis(df)

print("Key Performance Indicators:")
print("="*40)
for key, value in kpis.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")


Key Performance Indicators:
total_revenue: 5100479.43
total_orders: 10000
avg_order_value: 510.05
unique_customers: 2896
revenue_per_customer: 1761.22
aov_std: 552.02
aov_median: 331.14
revenue_skewness: 3.02
revenue_kurtosis: 14.19
avg_discount_rate: 10.04
total_discount_given: 565457.56
avg_delivery_days: 4.49
repeat_customer_rate: 0.89
avg_orders_per_customer: 3.45


In [24]:
# Statistical Analysis for Business Insights
def perform_statistical_analysis(df):
    print("\n" + "="*50)
    print("STATISTICAL ANALYSIS FOR BUSINESS INSIGHTS")
    print("="*50)

    # 1. Correlation Analysis
    print("\n1. CORRELATION ANALYSIS")
    print("-" * 30)
    numerical_cols = ['quantity', 'unit_price', 'total_price', 'final_price', 'discount_percent', 'delivery_days']
    correlation_matrix = df[numerical_cols].corr()
    print(correlation_matrix.round(3))

    # 2. Weekend vs Weekday Analysis
    print("\n2. WEEKEND vs WEEKDAY PERFORMANCE")
    print("-" * 40)
    df['is_weekend'] = pd.to_datetime(df['order_date']).dt.weekday >= 5
    weekend_stats = df.groupby('is_weekend').agg({
        'final_price': ['count', 'mean', 'sum'],
        'customer_id': 'nunique'
    }).round(2)
    print(weekend_stats)

    # Statistical significance test
    weekend_orders = df[df['is_weekend']]['final_price']
    weekday_orders = df[~df['is_weekend']]['final_price']
    t_stat, p_value = stats.ttest_ind(weekend_orders, weekday_orders)
    print(f"\nT-test results (Weekend vs Weekday AOV):")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"P-value: {p_value:.6f}")
    print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

    # 3. Category Performance Analysis
    print("\n3. CATEGORY PERFORMANCE ANALYSIS")
    print("-" * 40)
    category_stats = df.groupby('product_category').agg({
        'final_price': ['count', 'mean', 'sum', 'std'],
        'discount_percent': 'mean'
    }).round(2)
    print(category_stats)

    return correlation_matrix

correlation_matrix = perform_statistical_analysis(df)



STATISTICAL ANALYSIS FOR BUSINESS INSIGHTS

1. CORRELATION ANALYSIS
------------------------------
                  quantity  unit_price  total_price  final_price  \
quantity             1.000      -0.019        0.480        0.476   
unit_price          -0.019       1.000        0.761        0.757   
total_price          0.480       0.761        1.000        0.994   
final_price          0.476       0.757        0.994        1.000   
discount_percent    -0.007      -0.007       -0.008       -0.080   
delivery_days        0.018       0.003        0.010        0.010   

                  discount_percent  delivery_days  
quantity                    -0.007          0.018  
unit_price                  -0.007          0.003  
total_price                 -0.008          0.010  
final_price                 -0.080          0.010  
discount_percent             1.000          0.009  
delivery_days                0.009          1.000  

2. WEEKEND vs WEEKDAY PERFORMANCE
------------------------

In [25]:
#excel dashboard
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.chart import LineChart, BarChart, PieChart, Reference
import openpyxl

def create_excel_dashboard(df, kpis, monthly_data, product_performance):
    # Create workbook with multiple sheets
    wb = Workbook()

    # Remove default sheet
    wb.remove(wb.active)

    # 1. KPI Summary Sheet
    kpi_sheet = wb.create_sheet("KPI_Dashboard")

    # Headers
    kpi_sheet['A1'] = "Key Performance Indicators"
    kpi_sheet['A1'].font = Font(bold=True, size=16)
    kpi_sheet['A1'].fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")

    # KPI Data
    row = 3
    for key, value in kpis.items():
        kpi_sheet[f'A{row}'] = key.replace('_', ' ').title()
        if isinstance(value, float):
            kpi_sheet[f'B{row}'] = round(value, 2)
        else:
            kpi_sheet[f'B{row}'] = value
        row += 1

    # 2. Monthly Trends Sheet
    monthly_sheet = wb.create_sheet("Monthly_Trends")

    # Add monthly data
    monthly_sheet['A1'] = "Month"
    monthly_sheet['B1'] = "Total Orders"
    monthly_sheet['C1'] = "Total Revenue"
    monthly_sheet['D1'] = "AOV"
    monthly_sheet['E1'] = "Growth %"

    for idx, row_data in monthly_data.iterrows():
        excel_row = idx + 2
        monthly_sheet[f'A{excel_row}'] = row_data['month']
        monthly_sheet[f'B{excel_row}'] = row_data['total_orders']
        monthly_sheet[f'C{excel_row}'] = row_data['total_revenue']
        monthly_sheet[f'D{excel_row}'] = row_data['avg_order_value']
        monthly_sheet[f'E{excel_row}'] = row_data['revenue_growth_pct'] if not pd.isna(row_data['revenue_growth_pct']) else 0

    # 3. Product Performance Sheet
    product_sheet = wb.create_sheet("Product_Performance")

    # Add product data
    for idx, col in enumerate(['Category', 'Orders', 'Revenue', 'AOV', 'Revenue Share %']):
        product_sheet.cell(row=1, column=idx+1, value=col)

    for idx, row_data in product_performance.iterrows():
        excel_row = idx + 2
        product_sheet[f'A{excel_row}'] = row_data['product_category']
        product_sheet[f'B{excel_row}'] = row_data['total_orders']
        product_sheet[f'C{excel_row}'] = row_data['total_revenue']
        product_sheet[f'D{excel_row}'] = row_data['avg_order_value']
        product_sheet[f'E{excel_row}'] = row_data['revenue_share_pct']

    # Save Excel file
    wb.save('Meesho_KPI_Dashboard.xlsx')
    print("Excel dashboard created: Meesho_KPI_Dashboard.xlsx")

# Create Excel dashboard
create_excel_dashboard(df, kpis, monthly_data, product_performance)


Excel dashboard created: Meesho_KPI_Dashboard.xlsx


In [26]:
# Create advanced Excel analysis with formulas
def create_advanced_excel_analysis(df):
    # Prepare data for Excel pivot analysis
    excel_data = df.copy()
    excel_data['order_month'] = pd.to_datetime(excel_data['order_date']).dt.to_period('M').astype(str)
    excel_data['order_weekday'] = pd.to_datetime(excel_data['order_date']).dt.day_name()
    excel_data['revenue_bucket'] = pd.cut(excel_data['final_price'],
                                         bins=[0, 500, 1000, 2000, float('inf')],
                                         labels=['Low', 'Medium', 'High', 'Premium'])

    # Save to Excel for pivot table creation
    with pd.ExcelWriter('Meesho_Advanced_Analysis.xlsx', engine='openpyxl') as writer:
        excel_data.to_excel(writer, sheet_name='Raw_Data', index=False)

        # Monthly summary
        monthly_summary = excel_data.groupby('order_month').agg({
            'final_price': ['sum', 'mean', 'count'],
            'customer_id': 'nunique',
            'discount_amount': 'sum'
        }).round(2)
        monthly_summary.to_excel(writer, sheet_name='Monthly_Summary')

        # Category analysis
        category_analysis = excel_data.groupby(['product_category', 'customer_city']).agg({
            'final_price': ['sum', 'count'],
            'customer_id': 'nunique'
        }).round(2)
        category_analysis.to_excel(writer, sheet_name='Category_City_Analysis')

    print("Advanced Excel analysis created: Meesho_Advanced_Analysis.xlsx")

create_advanced_excel_analysis(df)


Advanced Excel analysis created: Meesho_Advanced_Analysis.xlsx


In [27]:
# Save this as dashboard.py
dashboard_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
import numpy as np
from datetime import datetime, timedelta

# Page configuration
st.set_page_config(
    page_title="Meesho Business Analytics Dashboard",
    page_icon="📊",
    layout="wide"
)

# Load data
@st.cache_data
def load_data():
    conn = sqlite3.connect('meesho_analytics.db')
    df = pd.read_sql_query("SELECT * FROM orders", conn)
    df['order_date'] = pd.to_datetime(df['order_date'])
    conn.close()
    return df

df = load_data()

# Sidebar filters
st.sidebar.header("Filters")
date_range = st.sidebar.date_input(
    "Select Date Range",
    value=(df['order_date'].min(), df['order_date'].max()),
    min_value=df['order_date'].min(),
    max_value=df['order_date'].max()
)

categories = st.sidebar.multiselect(
    "Select Categories",
    options=df['product_category'].unique(),
    default=df['product_category'].unique()
)

cities = st.sidebar.multiselect(
    "Select Cities",
    options=df['customer_city'].unique(),
    default=df['customer_city'].unique()
)

# Filter data
filtered_df = df[
    (df['order_date'].dt.date >= date_range[0]) &
    (df['order_date'].dt.date <= date_range[1]) &
    (df['product_category'].isin(categories)) &
    (df['customer_city'].isin(cities))
]

# Main dashboard
st.title("📊 Meesho Business Analytics Dashboard")
st.markdown("---")

# KPI Metrics Row
col1, col2, col3, col4, col5 = st.columns(5)

with col1:
    total_revenue = filtered_df['final_price'].sum()
    st.metric("Total Revenue", f"₹{total_revenue:,.0f}")

with col2:
    total_orders = len(filtered_df)
    st.metric("Total Orders", f"{total_orders:,}")

with col3:
    aov = filtered_df['final_price'].mean()
    st.metric("Average Order Value", f"₹{aov:.0f}")

with col4:
    unique_customers = filtered_df['customer_id'].nunique()
    st.metric("Unique Customers", f"{unique_customers:,}")

with col5:
    repeat_rate = (filtered_df.groupby('customer_id').size() > 1).mean() * 100
    st.metric("Repeat Customer Rate", f"{repeat_rate:.1f}%")

st.markdown("---")

# Charts Row 1
col1, col2 = st.columns(2)

with col1:
    # Monthly Revenue Trend
    monthly_revenue = filtered_df.groupby(filtered_df['order_date'].dt.to_period('M')).agg({
        'final_price': 'sum',
        'order_id': 'count'
    }).reset_index()
    monthly_revenue['order_date'] = monthly_revenue['order_date'].astype(str)

    fig_monthly = px.line(monthly_revenue, x='order_date', y='final_price',
                         title='Monthly Revenue Trend',
                         labels={'final_price': 'Revenue (₹)', 'order_date': 'Month'})
    fig_monthly.update_layout(showlegend=False)
    st.plotly_chart(fig_monthly, use_container_width=True)

with col2:
    # Category Performance
    category_revenue = filtered_df.groupby('product_category')['final_price'].sum().sort_values(ascending=True)

    fig_category = px.bar(x=category_revenue.values, y=category_revenue.index,
                         orientation='h',
                         title='Revenue by Category',
                         labels={'x': 'Revenue (₹)', 'y': 'Category'})
    fig_category.update_layout(showlegend=False)
    st.plotly_chart(fig_category, use_container_width=True)

# Charts Row 2
col1, col2 = st.columns(2)

with col1:
    # City Performance
    city_performance = filtered_df.groupby('customer_city').agg({
        'final_price': 'sum',
        'customer_id': 'nunique'
    }).sort_values('final_price', ascending=False).head(8)

    fig_city = px.scatter(city_performance, x='customer_id', y='final_price',
                         hover_data=['final_price'],
                         title='City Performance: Revenue vs Customers',
                         labels={'customer_id': 'Unique Customers', 'final_price': 'Revenue (₹)'})

    for i, city in enumerate(city_performance.index):
        fig_city.add_annotation(
            x=city_performance.loc[city, 'customer_id'],
            y=city_performance.loc[city, 'final_price'],
            text=city,
            showarrow=True,
            arrowhead=2
        )

    st.plotly_chart(fig_city, use_container_width=True)

with col2:
    # Payment Method Distribution
    payment_dist = filtered_df['payment_method'].value_counts()

    fig_payment = px.pie(values=payment_dist.values, names=payment_dist.index,
                        title='Payment Method Distribution')
    st.plotly_chart(fig_payment, use_container_width=True)

# Detailed Analysis Section
st.markdown("---")
st.header("📈 Detailed Analysis")

tab1, tab2, tab3 = st.tabs(["Customer Segmentation", "Product Analysis", "Time Analysis"])

with tab1:
    # Customer Segmentation
    customer_metrics = filtered_df.groupby('customer_id').agg({
        'final_price': ['sum', 'count', 'mean'],
        'order_date': 'max'
    }).round(2)

    customer_metrics.columns = ['total_spent', 'order_count', 'avg_order_value', 'last_order']
    customer_metrics['days_since_last_order'] = (datetime.now() - pd.to_datetime(customer_metrics['last_order'])).dt.days

    # Segmentation logic
    def segment_customers(row):
        if row['total_spent'] >= 5000 and row['order_count'] >= 5:
            return 'High Value'
        elif row['total_spent'] >= 2000 and row['order_count'] >= 3:
            return 'Medium Value'
        elif row['days_since_last_order'] <= 30:
            return 'Recent Active'
        elif row['days_since_last_order'] > 90:
            return 'At Risk'
        else:
            return 'Regular'

    customer_metrics['segment'] = customer_metrics.apply(segment_customers, axis=1)
    segment_summary = customer_metrics.groupby('segment').agg({
        'total_spent': ['count', 'mean'],
        'order_count': 'mean',
        'avg_order_value': 'mean'
    }).round(2)

    st.subheader("Customer Segmentation Analysis")
    st.dataframe(segment_summary)

    # Segment distribution pie chart
    segment_dist = customer_metrics['segment'].value_counts()
    fig_segments = px.pie(values=segment_dist.values, names=segment_dist.index,
                         title='Customer Segment Distribution')
    st.plotly_chart(fig_segments, use_container_width=True)

with tab2:
    # Product Analysis
    st.subheader("Product Category Performance")

    product_analysis = filtered_df.groupby('product_category').agg({
        'final_price': ['sum', 'count', 'mean'],
        'discount_percent': 'mean',
        'customer_id': 'nunique'
    }).round(2)

    product_analysis.columns = ['total_revenue', 'total_orders', 'avg_order_value', 'avg_discount', 'unique_customers']
    product_analysis['revenue_per_customer'] = (product_analysis['total_revenue'] / product_analysis['unique_customers']).round(2)

    st.dataframe(product_analysis.sort_values('total_revenue', ascending=False))

    # Discount vs Revenue analysis
    fig_discount = px.scatter(product_analysis, x='avg_discount', y='total_revenue',
                             size='total_orders', hover_name=product_analysis.index,
                             title='Discount Rate vs Revenue by Category')
    st.plotly_chart(fig_discount, use_container_width=True)

with tab3:
    # Time Analysis
    st.subheader("Time-based Analysis")

    # Weekday performance
    filtered_df['weekday'] = filtered_df['order_date'].dt.day_name()
    weekday_performance = filtered_df.groupby('weekday').agg({
        'final_price': ['sum', 'count', 'mean']
    }).round(2)
    weekday_performance.columns = ['total_revenue', 'total_orders', 'avg_order_value']

    # Reorder by weekday
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_performance = weekday_performance.reindex(weekday_order)

    fig_weekday = px.bar(weekday_performance, x=weekday_performance.index, y='total_revenue',
                        title='Revenue by Day of Week')
    st.plotly_chart(fig_weekday, use_container_width=True)

    # Hour analysis (if hour data available)
    if 'order_hour' in filtered_df.columns:
        hourly_orders = filtered_df.groupby('order_hour')['final_price'].sum()
        fig_hourly = px.line(x=hourly_orders.index, y=hourly_orders.values,
                           title='Revenue by Hour of Day')
        st.plotly_chart(fig_hourly, use_container_width=True)

# Insights and Recommendations
st.markdown("---")
st.header("💡 Key Insights & Recommendations")

insights = [
    f"📊 **Revenue Performance**: Total revenue of ₹{total_revenue:,.0f} from {total_orders:,} orders",
    f"💰 **Customer Value**: Average order value is ₹{aov:.0f} with {repeat_rate:.1f}% repeat customers",
    f"🏆 **Top Category**: {category_revenue.index[-1]} generates highest revenue",
    f"🎯 **Growth Opportunity**: Focus on customer retention to improve repeat rate",
    f"📈 **Recommendation**: Implement targeted campaigns for 'At Risk' customer segment"
]

for insight in insights:
    st.markdown(insight)

# Footer
st.markdown("---")
st.markdown("*Dashboard created for Meesho Business Analyst Role - Data-driven insights for business growth*")
'''

# Save dashboard code to file
with open('dashboard.py', 'w') as f:
    f.write(dashboard_code)

print("Streamlit dashboard created! Run with: streamlit run dashboard.py")


Streamlit dashboard created! Run with: streamlit run dashboard.py


In [31]:
%pip install schedule


In [32]:
#date refresh automation
import schedule
import time
from datetime import datetime

def automated_data_refresh():
    """Automated function to refresh data and regenerate reports"""
    print(f"Starting automated refresh at {datetime.now()}")

    # 1. Refresh data (in real scenario, this would connect to production DB)
    df_new = generate_ecommerce_data(1000)  # New daily data

    # 2. Append to existing database
    conn = sqlite3.connect('meesho_analytics.db')
    df_new.to_sql('orders', conn, if_exists='append', index=False)
    conn.close()

    # 3. Recalculate KPIs
    df_updated = pd.read_sql_query("SELECT * FROM orders", sqlite3.connect('meesho_analytics.db'))
    updated_kpis = calculate_kpis(df_updated)

    # 4. Generate updated Excel report
    # create_excel_dashboard(df_updated, updated_kpis, monthly_data, product_performance)

    # 5. Send email notification (placeholder)
    print("Data refresh completed successfully!")
    print(f"Updated KPIs: Total Revenue = ₹{updated_kpis['total_revenue']:,.2f}")

# Schedule automation (uncomment to run)
# schedule.every().day.at("06:00").do(automated_data_refresh)
# schedule.every().monday.at("09:00").do(automated_data_refresh)

print("Automation script ready. Uncomment scheduling lines to activate.")


Automation script ready. Uncomment scheduling lines to activate.


In [33]:
def kpi_monitoring_system(df, threshold_dict):
    """Monitor KPIs and generate alerts if thresholds are breached"""

    current_kpis = calculate_kpis(df)
    alerts = []

    # Check thresholds
    for kpi, threshold in threshold_dict.items():
        if kpi in current_kpis:
            current_value = current_kpis[kpi]

            if isinstance(threshold, dict):
                if 'min' in threshold and current_value < threshold['min']:
                    alerts.append(f"🚨 ALERT: {kpi} is below minimum threshold: {current_value:.2f} < {threshold['min']}")
                if 'max' in threshold and current_value > threshold['max']:
                    alerts.append(f"⚠️  WARNING: {kpi} is above maximum threshold: {current_value:.2f} > {threshold['max']}")

    return alerts

# Example thresholds
thresholds = {
    'avg_order_value': {'min': 200, 'max': 2000},
    'repeat_customer_rate': {'min': 0.3},
    'avg_discount_rate': {'max': 15}
}

alerts = kpi_monitoring_system(df, thresholds)
if alerts:
    print("KPI ALERTS:")
    for alert in alerts:
        print(alert)
else:
    print("✅ All KPIs within normal ranges")


✅ All KPIs within normal ranges


In [34]:
#generating Business Insights and actionable Recommendations
def generate_business_intelligence_report(df):
    """Generate comprehensive business intelligence with actionable insights"""

    print("\n" + "="*60)
    print("BUSINESS INTELLIGENCE REPORT - MEESHO ANALYTICS")
    print("="*60)

    # 1. Executive Summary
    print("\n📊 EXECUTIVE SUMMARY")
    print("-" * 30)
    total_revenue = df['final_price'].sum()
    total_orders = len(df)
    unique_customers = df['customer_id'].nunique()

    print(f"• Total Revenue: ₹{total_revenue:,.2f}")
    print(f"• Total Orders: {total_orders:,}")
    print(f"• Unique Customers: {unique_customers:,}")
    print(f"• Average Order Value: ₹{df['final_price'].mean():.2f}")
    print(f"• Revenue per Customer: ₹{total_revenue/unique_customers:.2f}")

    # 2. Key Performance Drivers
    print("\n🎯 KEY PERFORMANCE DRIVERS")
    print("-" * 35)

    # Top performing categories
    top_categories = df.groupby('product_category')['final_price'].sum().sort_values(ascending=False)
    print(f"• Top Revenue Category: {top_categories.index[0]} (₹{top_categories.iloc[0]:,.0f})")

    # City performance
    top_cities = df.groupby('customer_city')['final_price'].sum().sort_values(ascending=False)
    print(f"• Top Revenue City: {top_cities.index[0]} (₹{top_cities.iloc[0]:,.0f})")

    # Customer behavior insights
    repeat_customers = df.groupby('customer_id').size()
    repeat_rate = (repeat_customers > 1).sum() / len(repeat_customers) * 100
    print(f"• Repeat Customer Rate: {repeat_rate:.1f}%")

    # 3. Growth Opportunities
    print("\n📈 GROWTH OPPORTUNITIES")
    print("-" * 30)

    # Low performing segments
    low_performing_cities = top_cities.tail(3)
    print("• Cities with growth potential:")
    for city, revenue in low_performing_cities.items():
        print(f"  - {city}: ₹{revenue:,.0f}")

    # Discount analysis
    avg_discount = df['discount_percent'].mean()
    high_discount_orders = df[df['discount_percent'] > avg_discount]
    discount_impact = high_discount_orders['final_price'].mean() - df[df['discount_percent'] <= avg_discount]['final_price'].mean()

    print(f"• Average Discount Rate: {avg_discount:.1f}%")
    print(f"• Discount Impact on AOV: ₹{discount_impact:.0f}")

    # 4. Actionable Recommendations
    print("\n💡 ACTIONABLE RECOMMENDATIONS")
    print("-" * 40)

    recommendations = [
        "1. CUSTOMER RETENTION: Implement loyalty program to increase repeat rate from {:.1f}% to 40%+".format(repeat_rate),
        "2. CATEGORY EXPANSION: Focus marketing on underperforming categories with high-margin potential",
        "3. GEOGRAPHIC EXPANSION: Increase marketing spend in low-revenue cities with high growth potential",
        "4. PRICING OPTIMIZATION: Review discount strategy - current average of {:.1f}% may be impacting margins".format(avg_discount),
        "5. PERSONALIZATION: Implement customer segmentation for targeted campaigns",
        "6. INVENTORY MANAGEMENT: Focus on fast-moving categories: {}".format(', '.join(top_categories.head(3).index))
    ]

    for rec in recommendations:
        print(f"• {rec}")

    # 5. Predicted Impact
    print("\n📊 PREDICTED BUSINESS IMPACT")
    print("-" * 35)

    # Calculate potential improvements
    potential_repeat_improvement = (0.4 - repeat_rate/100) * unique_customers * df['final_price'].mean()
    potential_city_expansion = low_performing_cities.sum() * 0.5  # 50% growth in low cities

    print(f"• Potential Revenue from Improved Retention: ₹{potential_repeat_improvement:,.0f}")
    print(f"• Potential Revenue from City Expansion: ₹{potential_city_expansion:,.0f}")
    print(f"• Total Potential Revenue Increase: ₹{potential_repeat_improvement + potential_city_expansion:,.0f}")

    return {
        'recommendations': recommendations,
        'potential_revenue_increase': potential_repeat_improvement + potential_city_expansion,
        'key_metrics': {
            'total_revenue': total_revenue,
            'repeat_rate': repeat_rate,
            'avg_discount': avg_discount,
            'top_category': top_categories.index[0],
            'top_city': top_cities.index[0]
        }
    }

# Generate the comprehensive report
business_report = generate_business_intelligence_report(df)



BUSINESS INTELLIGENCE REPORT - MEESHO ANALYTICS

📊 EXECUTIVE SUMMARY
------------------------------
• Total Revenue: ₹5,100,479.43
• Total Orders: 10,000
• Unique Customers: 2,896
• Average Order Value: ₹510.05
• Revenue per Customer: ₹1761.22

🎯 KEY PERFORMANCE DRIVERS
-----------------------------------
• Top Revenue Category: Electronics (₹925,413)
• Top Revenue City: Delhi (₹664,519)
• Repeat Customer Rate: 88.5%

📈 GROWTH OPPORTUNITIES
------------------------------
• Cities with growth potential:
  - Chennai: ₹626,698
  - Ahmedabad: ₹618,136
  - Kolkata: ₹611,232
• Average Discount Rate: 10.0%
• Discount Impact on AOV: ₹-73

💡 ACTIONABLE RECOMMENDATIONS
----------------------------------------
• 1. CUSTOMER RETENTION: Implement loyalty program to increase repeat rate from 88.5% to 40%+
• 2. CATEGORY EXPANSION: Focus marketing on underperforming categories with high-margin potential
• 3. GEOGRAPHIC EXPANSION: Increase marketing spend in low-revenue cities with high growth potenti

In [35]:
import os

def create_project_structure():
    """Create organized project structure"""

    project_structure = {
        'data/': ['meesho_analytics.db'],
        'reports/': ['Meesho_KPI_Dashboard.xlsx', 'Meesho_Advanced_Analysis.xlsx'],
        'scripts/': ['data_generation.py', 'analytics.py', 'automation.py'],
        'dashboard/': ['dashboard.py'],
        'documentation/': ['README.md', 'business_requirements.md']
    }

    print("PROJECT STRUCTURE:")
    print("="*40)
    for folder, files in project_structure.items():
        print(f"📁 {folder}")
        for file in files:
            print(f"   📄 {file}")

    return project_structure

project_files = create_project_structure()


PROJECT STRUCTURE:
📁 data/
   📄 meesho_analytics.db
📁 reports/
   📄 Meesho_KPI_Dashboard.xlsx
   📄 Meesho_Advanced_Analysis.xlsx
📁 scripts/
   📄 data_generation.py
   📄 analytics.py
   📄 automation.py
📁 dashboard/
   📄 dashboard.py
📁 documentation/
   📄 README.md
   📄 business_requirements.md


In [36]:
# Create README documentation
readme_content = """
# Meesho KPI Optimization & Dashboard Automation Project

## Project Overview
This project demonstrates comprehensive business analytics capabilities for the Meesho Business Analyst role, showcasing advanced SQL, Excel, Python, and statistical analysis skills.

## Key Features
- ✅ Advanced SQL queries for business analytics
- ✅ Statistical analysis and correlation studies
- ✅ Interactive Streamlit dashboard
- ✅ Automated Excel reporting with multiple sheets
- ✅ KPI monitoring and alerting system
- ✅ Business intelligence with actionable insights
- ✅ Data automation and scheduling

## Technical Skills Demonstrated
1. **Advanced SQL**: Complex joins, window functions, CTEs, statistical queries
2. **Excel Integration**: Multi-sheet dashboards, pivot analysis, automated reporting
3. **Python Analytics**: Pandas, NumPy, statistical analysis, data visualization
4. **Business Intelligence**: KPI calculation, trend analysis, customer segmentation
5. **Dashboard Creation**: Interactive Streamlit application
6. **Automation**: Scheduled data refresh, alert systems

## Business Impact
- Identified key performance drivers across categories and cities
- Developed customer segmentation for targeted marketing
- Created automated monitoring for critical KPIs
- Generated actionable recommendations with projected revenue impact
- Built scalable analytics infrastructure for ongoing business insights

## Files Description
- `dashboard.py`: Interactive Streamlit dashboard
- `meesho_analytics.db`: SQLite database with business data
- `Meesho_KPI_Dashboard.xlsx`: Executive KPI dashboard
- `Meesho_Advanced_Analysis.xlsx`: Detailed analytical reports

## Running the Project
1. Install requirements: `pip install -r requirements.txt`
2. Run data generation: `python data_generation.py`
3. Launch dashboard: `streamlit run dashboard.py`
4. View Excel reports in the reports/ folder

## Key Insights Generated
- Revenue growth opportunities in underperforming cities
- Customer retention strategies to improve repeat rate
- Category performance optimization recommendations
- Discount strategy impact analysis

This project demonstrates the complete data → insight → action journey essential for business analyst roles.
"""

# Save README
with open('README.md', 'w') as f:
    f.write(readme_content)

print("✅ Complete KPI Optimization & Dashboard Automation Project Created!")
print("\nProject includes:")
print("- Synthetic e-commerce dataset (10,000+ orders)")
print("- Advanced SQL analytics queries")
print("- Statistical analysis and correlation studies")
print("- Interactive Streamlit dashboard")
print("- Automated Excel reporting")
print("- Business intelligence with actionable insights")
print("- Complete documentation and project structure")


✅ Complete KPI Optimization & Dashboard Automation Project Created!

Project includes:
- Synthetic e-commerce dataset (10,000+ orders)
- Advanced SQL analytics queries
- Statistical analysis and correlation studies
- Interactive Streamlit dashboard
- Automated Excel reporting
- Business intelligence with actionable insights
- Complete documentation and project structure


In [38]:
! streamlit run KPI_optimization.ipynb

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, not .ipynb.
For more information, please see https://docs.streamlit.io
